#Introduction
I play a critical part in analyzing data from historical events to draw lessons and inform how to address crises in the present and the future as a data scientist for a nonprofit that assists with disaster recovery. One of my specialties is estimating the likelihood that passengers and personnel would survive a crisis based on factors like age, gender, class, and cabin, all of which have a significant impact on the organization's preparedness and response.

I can find patterns and connections between the characteristics of the passengers and crew and their survival percentages by looking at historical data from tragedies like the sinking of the Titanic. With the use of this data, I can develop prediction models that project survival rates for individuals in like circumstances.

# Import libraries



In [ ]:
import pandas as pd
import numpy as np
import sklearn.model_selection
import sklearn.decomposition
import imblearn.over_sampling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn import svm
from sklearn.metrics import confusion_matrix
import numpy as np
from scipy.sparse import csr_matrix
%pip install fairlearn
from fairlearn.metrics import MetricFrame
from fairlearn.reductions import ExponentiatedGradient, DemographicParity
from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv("/content/Titanic-Dataset.csv")
data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


#data exploration

In [ ]:
#DESCRIBING OUR DATA
data.describe()

In [ ]:
#KNOWING THE SHAPE of our data we got 12 columns and 891 rows
data.shape

(891, 12)

In [ ]:
data.info()

we can know more about
The type of data in  each column.
The  non-null values.


#Showing discrimination

here we can see that we do have passenger who survived and the other who did not survived.
0: unsurvived passenger
1: survived passenger
and based on that we creat the foorloop to count how many females and males who got survived.
We can motice that the male survived male is less than survived woman by

In [ ]:
for any_gen in data['Sex'].unique():
    print(any_gen)

    x_df = data[ data['Sex']== any_gen ]
    survived = x_df[ x_df['Survived']==1 ]

    surv_perc = ( survived.shape[0] / x_df.shape[0] ) * 100

    print("Count: ", x_df.shape[0])
    print("Servived  : ", "%.2f" % surv_perc, '%')

    print("\n====\n")

male
Count:  577
Servived  :  18.89 %

====

female
Count:  314
Servived  :  74.20 %

====



replace the values in the Sex column ,which are male and female with 1 and 0 for bor both male and female.
and replace it in the origin datafram.

In [ ]:
data.head(5)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
# Splitting the main dataset into training and testing sets

x = data.drop(['Survived'], axis=1)
y = data['Survived']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y)
#printing the x_train and y_train.
print("df:", data.shape)
print("x_train", X_train.shape)
print("x_test:", X_test.shape)
print("y_train:", y_test.shape)
print("y_test:", y_test.shape)


df: (891, 11)
x_train (668, 10)
x_test: (223, 10)
y_train: (223,)
y_test: (223,)


i splited the data befor starting doing other featur engineering and other processes so separat definitely the training dataset and the test dataset,and not learn from each other as well

#Data Preprocessing and Feature Engineering


In [ ]:
data['Sex'].replace({'male': 1, 'female': 0}, inplace=True)

# Set 'PassengerId' as the index
data.set_index('PassengerId', inplace=True)


To address missing age values, I utilized the fillna() method to substitute them with the average age of all passengers.


In [ ]:

# Fill in missing values in the Age column of the training set with the mean
X_train['Age'].fillna(np.mean(X_train['Age']), inplace=True)

# Fill in missing values in the Age column of the testing set with the mean
X_test['Age'].fillna(np.mean(X_test['Age']), inplace=True)


I droped the Cabin here because i think it's useless for my prediction by using drop function.

In [ ]:
#here we can drop cabin
X_train.drop('Cabin', axis=1, inplace=True)
X_test.drop('Cabin', axis=1, inplace=True)

Defining the Features and the Target Label


#one hot encoding

I used one hot encoding to transfer categorical data to numerical dataset so our model could process it.because it is impringov the data's interpretability, which is beneficial for comprehending machine learning model output.

In [ ]:
# Creating one hot encoding object
encoder = sklearn.preprocessing.OneHotEncoder(handle_unknown='ignore')

# Fitting encoder on X_train
encoder.fit(X_train)

# Transforming X_train using the encoder and saving it back into X_train variable
X_train = encoder.transform(X_train)

# Transforming X_test using the encoder and saving it back into X_test variable
X_test = encoder.transform(X_test)

# Printing X_train shape to check the size
print("X_train:", X_train.shape)
print('y_train:',X_test.shape)

X_train: (668, 1524)
y_train: (223, 1524)


#Dealing with Imbalanced Classes




The training data's class imbalance is fixed by the algorithm using SMOTE, producing a more balanced dataset for the classification task.

In [ ]:
# Creating a SMOTE scaler object
sampler = imblearn.over_sampling.SMOTE(random_state=25)

# Fitting the scaler object on X_train and y_train and saving them in new variables
X_train_resampled, y_train_resampled = sampler.fit_resample(X_train, y_train)

# Checking the size of the resampled X_train and y_train
print("X_train_resampled size:",X_train_resampled.shape)
print("y_train_resampled size:",y_train_resampled.shape)


X_train_resampled size: (822, 1524)
y_train_resampled size: (822,)


Here i reduces the dimensionality of the training data by applying principal component analysis (PCA). A dimensionality reduction method called principal component analysis (PCA) finds the key features in a dataset and condenses them into a smaller set of new features. By decreasing the quantity of noise and redundancy in the data, this can aid in enhancing the performance of machine learning models.

In [ ]:
# Creating PCA object with 25 of random state
pca = sklearn.decomposition.PCA(random_state=25)

# We convert the sparse matrix X_train to a dense matrix by using toarray()
X_train_resampled = X_train_resampled.toarray()


# Fitting and transforming the resampled X_train
X_train_pca = pca.fit_transform(X_train_resampled)


# Transforming the resampled X_test
X_test_pca = pca.transform(X_test.toarray())

# Printing the size of resampled X_train and resampled X_test after dimensionality reduction.
print(X_train_pca.shape)


(822, 822)


In [ ]:
print(y_train.shape)


(668,)


I applied logistic regression and SVM to build the model. Then I trained both models with the x train data and used the x test data to make predictions.

In [ ]:
# Train logistic regression model
log_reg_model = LogisticRegression()
log_reg_model.fit(X_train_pca, y_train_resampled)

# Train SVM model
svm_model = SVC()
svm_model.fit(X_train_pca, y_train_resampled)

SVC()

In [ ]:
#Predicted model
y_pred_log_reg = log_reg_model.predict(X_test_pca)
y_pred_svm = svm_model.predict(X_test_pca)

Here i'm comparing the result of the prediction The logistic regression and support vector machine (SVM).
Based on these results, we can compare the performance of the two models. We can see that ogistic regression is a better model than SVM for this problem, as it has higher accuracy, precision, and F1 score. However, these results may vary depending on the data, the features, and the parameters of the models

In [ ]:
# Evaluate logistic regression
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
precision_log_reg = precision_score(y_test, y_pred_log_reg)
recall_log_reg = recall_score(y_test, y_pred_log_reg)
f1_log_reg = f1_score(y_test, y_pred_log_reg)

# Evaluate SVM
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_svm = f1_score(y_test, y_pred_svm)


In [ ]:
print("Logistic Regression:")
print(f"Accuracy: {accuracy_log_reg:.2f}")
print(f"Precision: {precision_log_reg:.2f}")
print(f"Recall: {recall_log_reg:.2f}")
print(f"F1 Score: {f1_log_reg:.2f}")
print("\n")

print("SVM:")
print(f"Accuracy: {accuracy_svm:.2f}")
print(f"Precision: {precision_svm:.2f}")
print(f"Recall: {recall_svm:.2f}")
print(f"F1 Score: {f1_svm:.2f}")
print("\n")

Logistic Regression:
Accuracy: 0.81
Precision: 0.72
Recall: 0.72
F1 Score: 0.72


SVM:
Accuracy: 0.82
Precision: 0.72
Recall: 0.75
F1 Score: 0.73




calculatin TP and TN for both models
The quantity of cases that are accurately anticipated to be negative is known as true negatives (TN).
The quantity of cases that are mistakenly projected as positive when they are actually negative is known as false positives (FP).
The quantity of cases that are mistakenly anticipated as negative when they are actually positive is known as false negatives (FN).
The quantity of cases that are accurately anticipated to be positive is known as true positives (TP).

In [ ]:
# For logistic regression model
tn_log, fp_log, fn_log, tp_log = confusion_matrix(y_test, y_pred_log_reg).ravel()

# For SVM model
tn_svm, fp_svm, fn_svm, tp_svm = confusion_matrix(y_test, y_pred_svm).ravel()

the result indicates that of the 223 cases, the logistic regression model predicted 54 positive and 127 negative cases correctly, and 21 positive and 21 negative cases inaccurately.and it`s the same for SVM

In [ ]:
print("Logistic Regression model confusion matrix:")
print("True negatives: ", tn_log)
print("False positives: ", fp_log)
print("False negatives: ", fn_log)
print("True positives: ", tp_log)

print("\nSVM model confusion matrix:")
print("True negatives: ", tn_svm)
print("False positives: ", fp_svm)
print("False negatives: ", fn_svm)
print("True positives: ", tp_svm)

Logistic Regression model confusion matrix:
True negatives:  127
False positives:  21
False negatives:  21
True positives:  54

SVM model confusion matrix:
True negatives:  126
False positives:  22
False negatives:  19
True positives:  56


After finding TP and TN we implement this methode which indicat the ratio of correctly predicted positive observations to the total predicted positives. It answers the question “What proportion of positive identifications was actually correct?”

In [ ]:
# For logistic regression model
tn_log, fp_log, fn_log, tp_log = confusion_matrix(y_test, y_pred_log_reg).ravel()

# For SVM model
tn_svm, fp_svm, fn_svm, tp_svm = confusion_matrix(y_test, y_pred_svm).ravel()
# Calculate the precision and recall for logistic regression
precision_log = tp_log / (tp_log + fp_log)
recall_log = tp_log / (tp_log + fn_log)

f1_score_log = 2 * (precision_log * recall_log) / (precision_log + recall_log)

# Calculate the precision and recall for SVM
precision_svm = tp_svm / (tp_svm + fp_svm)
recall_svm = tp_svm / (tp_svm + fn_svm)

f1_score_svm = 2 * (precision_svm * recall_svm) / (precision_svm + recall_svm)

print("Logistic Regression Model Performance:")
print("Precision:", precision_log)
print("Recall:", recall_log)
print("F1 Score:", f1_score_log)
print('====='*14)
print("SVM Model Performance:")
print("Precision:", precision_svm)
print("Recall:", recall_svm)
print("F1 Score:", f1_score_svm)

Logistic Regression Model Performance:
Precision: 0.7311827956989247
Recall: 0.8
F1 Score: 0.7640449438202247
SVM Model Performance:
Precision: 0.7083333333333334
Recall: 0.8
F1 Score: 0.7513812154696135


by using different accuracy mitrics we noticed that  both models have the same recall, but the logistic regression model has a higher precision and F1 score,which mean that LR the  predicted positive observations correctly is better than SVM

In [ ]:
# Calculate the weights based on the confusion matrix
tpr_majority_log = tp_log / (tp_log + fn_log)
tpr_minority_log = tp_log / (tp_log + fn_log)

# Calculate the difference in TPR between the majority group and the minority group for logistic regression
tpr_diff_log = abs(tpr_majority_log - tpr_minority_log)

# Assign weights to data points based on their sensitivity to bias for logistic regression
weights_log = 1 / (tpr_diff_log + 1)

# Refit the logistic regression model with the reweighted data
log_reg_model.fit(X_train, y_train, sample_weight=weights_log)

# Evaluate the reweighted logistic regression model on the test set
y_pred_reweighted_log = log_reg_model.predict(X_test)
tn_reweighted_log, fp_reweighted_log, fn_reweighted_log, tp_reweighted_log = confusion_matrix(y_test, y_pred_reweighted_log).ravel()

# Print the reweighted logistic regression model confusion matrix
print("Reweighted Logistic Regression model confusion matrix:")
print("True negatives:", tn_reweighted_log)
print("False positives:", fp_reweighted_log)
print("False negatives:", fn_reweighted_log)
print("True positives:", tp_reweighted_log)

Reweighted Logistic Regression model confusion matrix:
True negatives: 114
False positives: 24
False negatives: 24
True positives: 61


In [ ]:
# Number of correct predictions
correct_predictions = 123 + 59  # True negatives + True positives

# Total number of predictions
total_predictions = 123 + 16 + 25 + 59  # True negatives + False positives + False negatives + True positives

# Accuracy
accuracy = correct_predictions / total_predictions

print("Accuracy of reweighted logistic regression model:", accuracy)

Accuracy of reweighted logistic regression model: 0.8161434977578476


The result Accuracy of reweighted logistic regression model: 0.8 means that the model correctly predicted the outcome (either positive or negative) for 80% of the instances in the dataset.

In [ ]:

# Assuming tp_svm and fn_svm are numpy arrays representing the true positives and false negatives for each sapmle.
tp_svm = np.array([tp_svm])
fn_svm = np.array([fn_svm])

# Calculate the TPR for each group
tpr_majority = tp_svm / (tp_svm + fn_svm)
tpr_minority = tp_svm / (tp_svm + fn_svm)

# Calculate the difference in TPR between the majority group and the minority group
tpr_diff = abs(tpr_majority - tpr_minority)

# Convert weights_svm to a NumPy array with the same length as X_train
weights_svm = np.full((len(X_train_pca),), tpr_diff + 0.0001)

# Convert X_train to a sparse matrix
X_train_sparse = csr_matrix(X_train_pca)

# Refit the SVM model with the reweighted data
svm_model = svm.SVC()
svm_model.fit(X_train_sparse, y_train_resampled, sample_weight=weights_svm)

# Evaluate the reweighted SVM model on the test set
y_pred_reweighted_svm = svm_model.predict(X_test_pca)
tn_reweighted_svm, fp_reweighted_svm, fn_reweighted_svm, tp_reweighted_svm = confusion_matrix(y_test, y_pred_reweighted_svm).ravel()

# Print the reweighted SVM model confusion matrix
print("Reweighted SVM model confusion matrix:")
print("True negatives:", tn_reweighted_svm)
print("False positives:", fp_reweighted_svm)
print("False negatives:", fn_reweighted_svm)
print("True positives:", tp_reweighted_svm)

Reweighted SVM model confusion matrix:
True negatives: 118
False positives: 20
False negatives: 25
True positives: 60


From these results, it seems like my model is doing well at predicting negative instances (with 140 true negatives and 0 false positives), but it’s not doing well at predicting positive instances (with 0 true positives and 83 false negatives). This could indicate that your model is biased towards predicting negative instances, or it could be a sign that your model is not well-suited to this particular dataset or problem. You might want to consider trying a different model or adjusting the parameters of your SVM model to improve its performance.

In [ ]:
# Number of correct predictions
correct_predictions = 98 + 58  # True negatives + True positives

# Total number of predictions
total_predictions = 98 + 50 + 17 + 58  # True negatives + False positives + False negatives + True positives

# Accuracy
accuracy = correct_predictions / total_predictions

print("Accuracy of reweighted SVM model:", accuracy)

Accuracy of reweighted SVM model: 0.6995515695067265


# Add more hyper parapmeter tuning to improve the accuracy

In [ ]:
# Create a logistic regression model with L2 regularization
log_reg_reg = LogisticRegression(C=0.1, penalty='l2')

# Fit the model
log_reg_reg.fit(X_train, y_train)

# Evaluate the model
y_pred = log_reg_reg.predict(X_test)

In [ ]:
# List of regularization parameters and penalties
parameters = [0.1, 0.01, 0.001]
penalties = ['l2']

# Loop over each combination of parameters
for C in parameters:
    for penalty in penalties:
        # Create and fit the model
        log_reg_reg = LogisticRegression(C=C, penalty=penalty)
        log_reg_reg.fit(X_train_pca, y_train_resampled)

        # Predict the labels of the test set
        y_pred = log_reg_reg.predict(X_test_pca)

        # Calculate the accuracy
        accuracy = accuracy_score(y_test, y_pred_log_reg)

        # Print the results
        print(f"Logistic Regression model with C={C} and penalty={penalty}:")
        print(f"Accuracy: {accuracy}")

Logistic Regression model with C=0.1 and penalty=l2:
Accuracy: 0.8116591928251121
Logistic Regression model with C=0.01 and penalty=l2:
Accuracy: 0.8116591928251121
Logistic Regression model with C=0.001 and penalty=l2:
Accuracy: 0.8116591928251121


In [ ]:
# List of regularization parameters
parameters = [0.1, 1, 10]

# Loop over each regularization parameter
for C in parameters:
    # Create and fit the model
    svm_model = svm.SVC(C=C)
    svm_model.fit(X_train_pca, y_train_resampled)

    # Predict the labels of the test set
    y_pred_svm = svm_model.predict(X_test_pca)

    # Calculate the accuracy
    accuracy = accuracy_score(y_test, y_pred_svm)

    # Print the results
    print(f"SVM model with C={C}:")
    print(f"Accuracy: {accuracy}")

SVM model with C=0.1:
Accuracy: 0.7937219730941704
SVM model with C=1:
Accuracy: 0.7982062780269058
SVM model with C=10:
Accuracy: 0.7937219730941704


For the SVM model:

The model correctly predicted 73 true negatives and 25 true positives.
However, the model incorrectly predicted 32 false positives and 49 false negatives.

By using fairlearn liberary which is used to train and evaluate machine learning models in a way that ensures fairness across different demographic groups.But i created a copy of my dataset and i drop all the other columns because i found difficulty of having the same length of my targetlabels.

In [ ]:
# Create a copy of your data
data_copy = data.copy()

# Replace 'male' with 1 and 'female' with 0 in the 'Sex' column
data_copy['Sex'].replace({'male': 1, 'female': 0}, inplace=True)

# Define the features and target variable
X = data_copy['Sex']
y = data_copy['Survived']

# Split the data again
X_train_copy, X_test_copy, y_train_copy, y_test_copy = train_test_split(X, y)

# Reshape X_train_copy and X_test_copy to 2D arrays
X_train_copy = X_train_copy.values.reshape(-1, 1)
X_test_copy = X_test_copy.values.reshape(-1, 1)

# Create a copy of the sensitive_features array
sensitive_features_train_copy = X_train_copy
sensitive_features_test_copy = X_test_copy

# Fit the mitigators
mitigator_log_reg_copy = ExponentiatedGradient(log_reg_model, constraints=DemographicParity())
mitigator_log_reg_copy.fit(X_train_copy, y_train_copy, sensitive_features=sensitive_features_train_copy)

mitigator_svm_copy = ExponentiatedGradient(svm_model, constraints=DemographicParity())
mitigator_svm_copy.fit(X_train_copy, y_train_copy, sensitive_features=sensitive_features_train_copy)

# Make predictions
y_pred_mitigated_log_reg_copy = mitigator_log_reg_copy.predict(X_test_copy)
y_pred_mitigated_svm_copy = mitigator_svm_copy.predict(X_test_copy)

# Compute the fairness metrics
metric_frame_log_reg_copy = MetricFrame(metrics=accuracy_score, y_true=y_test_copy, y_pred=y_pred_mitigated_log_reg_copy, sensitive_features=sensitive_features_test_copy)
metric_frame_svm_copy = MetricFrame(metrics=accuracy_score, y_true=y_test_copy, y_pred=y_pred_mitigated_svm_copy, sensitive_features=sensitive_features_test_copy)


In [ ]:
# Print the accuracy of the mitigated logistic regression model
print("Accuracy of mitigated logistic regression model: ", metric_frame_log_reg_copy.overall)

# Print the accuracy of the mitigated SVM model
print("Accuracy of mitigated SVM model: ", metric_frame_svm_copy.overall)

Accuracy of mitigated logistic regression model:  0.5919282511210763
Accuracy of mitigated SVM model:  0.600896860986547


#Conclusion

 The mitigated SVM model has a slightly higher accuracy than the mitigated logistic regression model. However, as the Professor said a higher accuracy doesn’t always mean a better model. It’s essential to consider the balance between accuracy and fairness.

#Reference
M YASSER H (2021),Titanic Dataset.Available at : https://www.kaggle.com/datasets/yasserh/titanic-dataset (Accessed 09/12/2023)

In [ ]:
!jupyter nbconvert --to html /content/my_ethics_pro.ipynb

[NbConvertApp] Converting notebook /content/my_ethics_pro.ipynb to html
[NbConvertApp] Writing 827953 bytes to /content/my_ethics_pro.html


In [ ]:
!pip install pdfkit
